In [ ]:
# Train xgboost by epoch. 
# Start with default weights, train XGBoost, produce accuracy
# Run epoch again, but default weight will change, train XGBoost, product accuracy
# continue

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

import xgboost as xgb

import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv('./DATA/sensorless_data.csv')
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,TARGET
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


### Format input data

In [4]:
x_cols = list(data.columns[:-1])
X_data = data[x_cols].copy()
# Adding 0 for easy reshaping
X_data['F49'] = 0
X_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,0
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,0
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,0
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,0
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,0


In [81]:
len(X_data)

58509

In [82]:
X_data.shape

(58509, 49)

In [83]:
X = X_data.to_numpy().reshape((len(X_data),1,7,7))
X.shape

(58509, 1, 7, 7)

In [95]:
X = torch.tensor(X).float()
X.shape

C:\Users\thejj\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([58509, 1, 7, 7])

In [85]:
yt = torch.tensor(list(data['TARGET']-1))

In [86]:
yt

tensor([ 0,  0,  0,  ..., 10, 10, 10])

In [87]:
y = torch.zeros(yt.shape[0], 11)
y.scatter_(1, yt.unsqueeze(1), 1)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [88]:
y.unsqueeze(1).shape

torch.Size([58509, 1, 11])

### Randomizing dataset

In [89]:
idxs = np.arange(len(X))
samples = np.random.choice(idxs,size=10000)

X_rand = X[samples]
y_rand = y[samples]

### Combined model attempt

In [90]:
# Only contains layers up to flatten
# Xgboost will take us to final 11 logits (probailities)
# model = keras.models.Sequential([
#     keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
#     keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
#     keras.layers.Flatten()
# ])

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 256, (2,2), stride=1)
        self.conv2 = nn.Conv2d(256, 256, (2,2), stride=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        return x


cnn = CNN()

In [91]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

In [92]:
X_rand.shape

torch.Size([10000, 1, 7, 7])

In [94]:
X_rand[0]

tensor([[[-5.5173e-08, -5.9362e-06,  1.7998e-06,  1.7192e-06, -4.0898e-06,
          -1.1193e-05,  2.2568e-02],
         [ 2.2573e-02,  2.2572e-02,  9.5382e-03,  9.5423e-03,  9.5534e-03,
           8.0762e-04,  3.0720e-04],
         [ 5.3157e-04,  8.0442e-04,  2.5782e-04,  5.1103e-04,  1.7498e+00,
           1.7498e+00,  1.7498e+00],
         [ 1.7490e+00,  1.7489e+00,  1.7489e+00, -1.2353e-03, -1.1727e+00,
          -6.2496e-01, -4.9502e-03],
         [-1.7652e-01, -1.0123e-01,  3.3924e-03,  3.3943e-03,  3.3822e-03,
          -6.4078e-05, -6.3066e-05],
         [-4.2173e-05, -5.2963e-01,  1.8750e+01,  1.8028e+01, -4.6928e-01,
           1.9009e+00,  6.4626e+00],
         [-1.4981e+00, -1.4980e+00, -1.4980e+00, -1.4996e+00, -1.4996e+00,
          -1.4996e+00,  0.0000e+00]]], dtype=torch.float64)

In [108]:
optimizer.zero_grad()
features = cnn(X_rand.float())

In [110]:
features.detach().numpy()

(10000, 6400)

In [113]:
y_rand.shape

torch.Size([10000, 11])

In [119]:
features.detach().numpy()[0]

array([0.       , 0.       , 0.       , ..., 0.7548722, 0.       ,
       0.       ], dtype=float32)

In [133]:
params = {'objective':'multi:softprob', 'num_class':11}
labels = [list(r).index(0) for r in y_rand.numpy()]

dtrain = xgb.DMatrix(features.detach().numpy(), label=labels)
bst = xgb.train(params, dtrain)
logits = bst.predict(dtrain)

[20:24:47] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [170]:
features[:,:11] = torch.tensor(logits)

In [176]:
features[:,:11].shape

torch.Size([10000, 11])

In [178]:
torch.tensor(labels).shape

torch.Size([10000])

In [173]:
loss = criterion(features[:,:11], torch.tensor(labels))
loss.backward()
optimizer.step()

RuntimeError: Index out of range
Exception raised from should_compute_output at ..\torch/csrc/autograd/function.h:230 (most recent call first):
00007FF8F71A6AD600007FF8F71A6A70 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FF8AFDAB93000007FF8AFDAB7C0 torch_cpu.dll!torch::autograd::Node::should_compute_output [<unknown file> @ <unknown line number>]
00007FF8AFD589AF00007FF8AFD58910 torch_cpu.dll!torch::autograd::generated::AsStridedBackward::apply [<unknown file> @ <unknown line number>]
00007FF8AFD4D96700007FF8AFD4D760 torch_cpu.dll!torch::autograd::Node::operator() [<unknown file> @ <unknown line number>]
00007FF8B02259F900007FF8B02254A0 torch_cpu.dll!torch::autograd::Engine::add_thread_pool_task [<unknown file> @ <unknown line number>]
00007FF8B02265A500007FF8B0226290 torch_cpu.dll!torch::autograd::Engine::evaluate_function [<unknown file> @ <unknown line number>]
00007FF8B022B52C00007FF8B022B220 torch_cpu.dll!torch::autograd::Engine::thread_main [<unknown f

In [ ]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    
    

    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize
    outputs = cnn(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

print('Finished Training')